# [모듈 4.1] 평가 및 조건 단계 개발 

이 노트북은 아래와 같은 목차로 진행 됩니다. 전체를 모두 실행시에 완료 시간은 약 5분-10분 소요 됩니다.

- 1. 모델 평가 및 조건 단계 개요
- 2. 기본 라이브러리 로딩
- 3. 모델 아피텍트 위치 확인
- 4. 모델 빌딩 파이프라인 의 스텝(Step) 생성
- 5. 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의 및 실행
- 6. 파이프라인 실행 확인 하기
    
---

# 1. 모델 평가 및 조건 단계 개요

- 모델 처리 및 조건 단계의 개발자 가이드 
    - [모델 처리 스텝](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing)
    - [모델 조건 단계](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/build-and-manage-steps.html#step-type-condition)


# 2. 기본 라이브러리 로딩

세이지 메이커 관련 라이브러리를 로딩 합니다.

In [1]:
import boto3
import sagemaker
import pandas as pd
import os

#region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
sm_client = boto3.client("sagemaker")


## 2.1 노트북 변수 로딩


In [2]:
%store -r

# 3. 모델 아피텍트 위치 확인

In [3]:
print("train_model_artifact: \n", train_model_artifact)

train_model_artifact: 
 s3://sagemaker-us-east-1-051065130547/sagemaker-webinar-pipeline-advanced/training_jobs/pipelines-twg45ptdqzid-Fraud-Advance-Train-lzWNxOzDEB/output/model.tar.gz


# 4. 모델 빌딩 파이프라인 의 스텝(Step) 생성


## 4.1 모델 빌딩 파이프라인 변수 생성

본 노트북에서 사용하는 파라미터는 다음과 같습니다.

* `processing_instance_type` - 프로세싱 작업에서 사용할 `ml.*` 인스턴스 타입
* `processing_instance_count` - 프로세싱 작업에서 사용할 `ml.*` 인스턴스 갯수
* `input_data` - 입력데이터에 대한 S3 버킷 URI



파이프라인의 각 스텝에서 사용할 변수를 파라미터 변수로서 정의 합니다.


In [5]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.m5.xlarge"
)
train_model_artifact_path = ParameterString(
    name="TrainModelArtifactPath",
    default_value= train_model_artifact
)
test_preproc_data_path = ParameterString(
    name="TestPreprocDataPath",
    default_value= test_preproc_data_uri
)



## 4.2 SKLearn Processor 생성

SKLearn Processor 생성시에 인자가 필요 합니다.



In [6]:
#from sagemaker.processing import ScriptProcessor
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

script_eval = SKLearnProcessor(
                             framework_version= "0.23-1",
                             role=role,
                             instance_type=processing_instance_type,
                             instance_count= processing_instance_count,
                             base_job_name="script-fraud-scratch-eval",
                                    )



## 4.3 Property 파일 정의

- PropertyFile 은 step_eval 단계가 실행 한 후에 모델 평가 지표의 결과 파일 내용을 정의하는데 사용 됩니다.

```
형식:
<property_file_instance> = PropertyFile(
    name="<property_file_name>",
    output_name="<processingoutput_output_name>",
    path="<path_to_json_file>"
)
예시:
evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation",
    path="evaluation.json"
)
```


- 위의 PropertyFile 의  output_name="evaluation" 이고, 파일 이름은 evaluation.json" 라는 것을 의미 합니다. "evaluation.json" 파일 안에는 아래의 값이 저장이 됩니다.

```
    report_dict = {
        "binary_classification_metrics": {
            "auc": {
                "value": <roc_score>,
                "standard_deviation" : "NaN",
            },
        },
    }
```


- 최종적으로 evaluation.json 안의 \<roc_score\> 값이 추후 (조건 스텝) 에 사용이 됩니다.
- step_eval 이 실행이 되면 `evaluation.json` 이 S3에 저장이 됩니다.



#### 참고
- 참고 자료: [Property Files and JsonGet](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/build-and-manage-propertyfile.html)

In [7]:
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import ProcessingStep

from sagemaker.workflow.properties import PropertyFile


evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation",
    path="evaluation.json"
)



## 4.4 모델 평가 스텝 정의

In [8]:
step_eval = ProcessingStep(
    name="Fraud-Advance-Eval",
    processor=script_eval,
    inputs=[
        ProcessingInput(
            source= train_model_artifact_path,
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source= test_preproc_data_path,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="src/evaluation.py",
    property_files=[evaluation_report],
)

## 4.5 조건  단계 정의
- 조건 단계에서 사용하는 ConditionLessThanOrEqualTo 에서 evaluation.json 을 로딩하여 내용을 확인

```

형식:
cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step=step_eval,
        property_file=<property_file_instance>,
        json_path="test_metrics.roc.value",
    ),
    right=6.0
)

에시:
cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step=step_eval,
        property_file=evaluation_report,
        json_path="binary_classification_metrics.auc.value",
    ),
    right=6.0
)


```

- property_file=evaluation_report 는 위의 모델 평가 스텝에서 정의한  PropertyFile-evaluation_report 를 사용합니다. evaluation_report 에서 정의한 evaluation.json 파일 안의 "binary_classification_metrics.auc.value" 의 값을 사용한다는 것을 의미 합니다.


In [9]:
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)


cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step=step_eval,
        property_file=evaluation_report,
        json_path="binary_classification_metrics.auc.value",
    ),
    right=6.0
)

step_cond = ConditionStep(
    name="Fraud-Advance-Condition",
    conditions=[cond_lte],
    if_steps=[],    
    else_steps=[], 
)

The class JsonGet has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


# 5. 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의 및 실행


이제 지금까지 생성한 단계들을 하나의 파이프라인으로 조합하고 실행하도록 하겠습니다.

파이프라인은 name, parameters, steps 속성이 필수적으로 필요합니다. 
여기서 파이프라인의 이름은 (account, region) 조합에 대하여 유일(unique))해야 합니다.
우리는 또한 여기서 Experiment 설정을 추가 하여, 실험에 등록 합니다.

주의:

- 정의에 사용한 모든 파라미터가 존재해야 합니다.
- 파이프라인으로 전달된 단계(step)들은 실행순서와는 무관합니다. SageMaker Pipeline은 단계가 실행되고 완료될 수 있도록 의존관계를를 해석합니다.

## 5.1 파이프라인 정의
파이프라인 정의시에 아래 3개의 인자를 제공합니다.
- 파이프라인 이름
- 파이프라인 파라미터
- 파이프라인 실험 설정
- 스텝 정의 

In [10]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = project_prefix + "-Eval-Cond-Step"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type, 
        processing_instance_count,
        train_model_artifact_path,
        test_preproc_data_path, 
    ],
   steps=[step_eval, step_cond],
)

## 5.2 파이프라인 정의 확인
위에서 정의한 파이프라인 정의는 Json 형식으로 정의 되어 있습니다.

In [11]:
import json


definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'ProcessingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'ProcessingInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'TrainModelArtifactPath',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-us-east-1-051065130547/sagemaker-webinar-pipeline-advanced/training_jobs/pipelines-twg45ptdqzid-Fraud-Advance-Train-lzWNxOzDEB/output/model.tar.gz'},
  {'Name': 'TestPreprocDataPath',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-us-east-1-051065130547/sagemaker-webinar-pipeline-advanced/preporc/test.csv'}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'Fraud-Advance-Eval',
   'Type': 'Processing',
   'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType': {'Get': 'Parameters.ProcessingInstanceType'},
      'InstanceCount': {'Get':

## 5.3 파이프라인 정의를 제출하고 실행하기 

파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [12]:
pipeline.upsert(role_arn=role)
execution = pipeline.start()

In [13]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:051065130547:pipeline/sagemaker-webinar-pipeline-advanced-eval-cond-step',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:051065130547:pipeline/sagemaker-webinar-pipeline-advanced-eval-cond-step/execution/0fi1kuucungy',
 'PipelineExecutionDisplayName': 'execution-1647866257006',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2022, 3, 21, 12, 37, 36, 862000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 3, 21, 12, 37, 36, 862000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': '5cc58385-816c-4cb5-9eea-6146976c146a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5cc58385-816c-4cb5-9eea-6146976c146a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '465',
   'date': 'Mon, 21 Mar 2022 12:37:36 GMT'},
  'RetryAttempts': 0}}

## 5.4 파이프라인 실행 기다리기

In [14]:
execution.wait()

## 5.5 파이프라인 실행 단계 기록 보기

In [15]:
execution.list_steps()

[{'StepName': 'Fraud-Advance-Condition',
  'StartTime': datetime.datetime(2022, 3, 21, 12, 42, 46, 431000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 3, 21, 12, 42, 46, 854000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'Condition': {'Outcome': 'True'}}},
 {'StepName': 'Fraud-Advance-Eval',
  'StartTime': datetime.datetime(2022, 3, 21, 12, 37, 37, 845000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 3, 21, 12, 42, 45, 468000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:051065130547:processing-job/pipelines-0fi1kuucungy-fraud-advance-eval-wy4eo0oxka'}}}]

# 6. 파이프라인 실행 확인 하기


## 6.1 모델 평가 단계 결과 확인 하기
모델 평가의 결과로서 evaluation.json 이 S3에 저장 됩니다.

![eval_step_result.png](img/eval_step_result.png)

## 6.2 조건 스텝 결과 확인 하기
- 조건 스템의 결과로서 `True` 제공 되었습니다.

![condition-result.png](img/condition-result.png)